In [1]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.decomposition import PCA
import numpy as np 
import GPy
import torch
from __future__ import absolute_import, division, print_function
import torch
from torch.distributions import constraints
from torch.nn import Parameter
import pyro
from pyro.contrib.gp.util import Parameterized
import pyro.distributions as dist
import pyro.infer as infer
import pyro.optim as optim
from pyro.params import param_with_module_name
import pyro.contrib.gp as gp
import pyro.distributions as dist
import matplotlib.pyplot as plt
pyro.clear_param_store()
import tensorflow as tf
import numpy as np 
import pandas as pd
import numpy
import math
import cProfile
import time
import GPy
import numpy as np
from matplotlib import pyplot as plt


#import pymc
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.mplot3d import Axes3D
from scipy import stats
from scipy.special import gamma
from GPy.core.parameterization.variational import NormalPosterior, NormalPrior

import pyro.contrib.gp.models.vsgp as vsgp
from GPy.util.linalg import pdinv
from pyro.infer.mcmc.hmc import HMC
from pyro.infer.mcmc import MCMC
from matplotlib import pyplot
pyro.clear_param_store()
import time
torch.manual_seed(12)
pyro.set_rng_seed(101)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.decomposition import PCA
import numpy as np 
import gpflow
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from __future__ import print_function
import gpflow
from gpflow import kernels
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
gpflow.settings.numerics.quadrature = 'error'  # throw error if quadrature is used for kernel expectations

In [2]:
X = np.load('X.npy')
Y = np.load('Y.npy')
N = X.shape[0]
sample_mean = np.mean(X[:N,:])
sample_std = np.std(X[:N,:])
sample_norm = (X[:N,:] - sample_mean)/sample_std # normalisation
X_flat = np.array(sample_norm).reshape((2062, 64*64))

In [3]:
M = 100  # number of inducing pts
N = X_flat.shape[0]
Q=10
X_mean = gpflow.models.PCA_reduce(X_flat, Q) # Initialise via PCA
Z = np.random.permutation(X_mean.copy())[:M]
k = (kernels.RBF(Q, ARD=True))
m4 = gpflow.models.BayesianGPLVM(X_mean=X_mean, X_var=0.1*np.ones((N, Q)), Y= tf.cast(X_flat, tf.float64),
                                kern=k, M=M, Z=Z)

In [4]:
m4.likelihood.variance= 0.2953218115879997

In [5]:
m4.compile()
o = gpflow.train.AdamOptimizer(0.01)
#o.minimize(m4, maxiter=100) # start near MAP

In [6]:
s = gpflow.train.HMC()
samples = s.sample(m4, 100, epsilon=0.12, lmax=20, lmin=5, thin=5, logprobs=True)

In [7]:
X_loc = np.average(samples['BayesianGPLVM/X_mean'], axis=0)
X_var = np.average(samples['BayesianGPLVM/X_var'], axis=0)
log_prob = samples.logprobs

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_loc, Y, test_size=0.3, random_state=42)

In [9]:
clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(100, 100, 100, 100), random_state=1)

In [10]:
names = ["Nearest Neighbors", "Gaussian Process",
         "Decision Tree", "Random Forest", "AdaBoost",
         "Naive Bayes", "QDA","Linear SVC"]

In [11]:
classifiers = [
    KNeighborsClassifier(3),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1,random_state=42),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis(),
    SVC(kernel="linear", C=0.025)]

In [12]:
parameters = [
              {'C': [1, 10, 100], 'kernel': ['linear']},
              {'C': [1, 10, 100], 'gamma': [0.1,0.001, 0.0001], 'kernel': ['rbf','sigmoid','poly']},
             ]

In [13]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV

In [14]:
model = svm.SVC()
clf3 = GridSearchCV(model, parameters)
unique_rows = np.unique(y_train, axis=0)
i=0
d={}
for i in range(len(unique_rows)):
    d[str(unique_rows[i])]=i
temp_train=[]
for i in y_train:
    temp_train.append(d[str(i)])

In [15]:
clf3.fit(X_train, np.array(temp_train))

GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'C': [1, 10, 100], 'kernel': ['linear']}, {'C': [1, 10, 100], 'gamma': [0.1, 0.001, 0.0001], 'kernel': ['rbf', 'sigmoid', 'poly']}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [16]:
temp_test=[]
unique_rows = np.unique(y_test, axis=0)
i=0
d={}
for i in range(len(unique_rows)):
    d[str(unique_rows[i])]=i
for i in y_test:
    temp_test.append(d[str(i)])

In [17]:
y_hat_reduced = clf3.predict(X_test)

In [18]:
print("accuracy: " + str(accuracy_score(np.array(temp_test), y_hat_reduced)))

accuracy: 0.617124394184168


In [19]:
clf.fit(X_train,y_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 100, 100, 100), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [20]:
y_hat_reduced = clf.predict(X_test)

In [21]:
 print("accuracy: " + str(accuracy_score(y_test, y_hat_reduced)))

accuracy: 0.4894991922455573


In [22]:
for i in range(8):
    print(names[i])
    classifiers[i].fit(X_train, np.array(temp_train))
    score = classifiers[i].score(X_test, np.array(temp_test))
    print(names[i]+':'+str(score))

Nearest Neighbors
Nearest Neighbors:0.5105008077544426
Gaussian Process
Gaussian Process:0.09531502423263329
Decision Tree
Decision Tree:0.36348949919224555
Random Forest
Random Forest:0.4297253634894992
AdaBoost
AdaBoost:0.37479806138933763
Naive Bayes
Naive Bayes:0.4911147011308562
QDA
QDA:0.5815831987075929
Linear SVC
Linear SVC:0.5331179321486268


In [23]:
import pickle
model={}
model[0]=s
pickle.dump(model, open( "savefit-MCMCLatent5.p", "wb" ) )

In [24]:
import pickle
model={}
model[0]=samples
pickle.dump(model, open( "savefit-MCMCsamples.p", "wb" ) )